In [1]:
import camelot
import pandas as pd

In [2]:
tables = camelot.read_pdf("payroll_report.pdf",pages='all', flavor='lattice')

c:\Users\Biky Roy\AppData\Local\Programs\Python\Python310\lib\site-packages\camelot\utils.py:740: UserWarning:   (569.14608, 571.64112) does not lie in column range (45.11043823685006, 568.565392986699)
  warnings.warn(
c:\Users\Biky Roy\AppData\Local\Programs\Python\Python310\lib\site-packages\camelot\utils.py:740: UserWarning:   (567.71088, 570.20592) does not lie in column range (45.11043823685006, 567.1256831922612)
  warnings.warn(
c:\Users\Biky Roy\AppData\Local\Programs\Python\Python310\lib\site-packages\camelot\utils.py:740: UserWarning:   (568.42848, 570.92352) does not lie in column range (45.11043823685006, 567.6055864570737)
  warnings.warn(
c:\Users\Biky Roy\AppData\Local\Programs\Python\Python310\lib\site-packages\camelot\utils.py:740: UserWarning:   (569.1240000000001, 571.6190400000002) does not lie in column range (45.11043823685006, 568.6853688029021)
  warnings.warn(
c:\Users\Biky Roy\AppData\Local\Programs\Python\Python310\lib\site-packages\camelot\utils.py:740: Use

In [5]:
tables = list(map(lambda x:x.df, tables))

In [6]:
tables[0]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,Age,Number of new EPF subscribers during the period,,,,,Number of members that ceased subscribing duri...,,,,,Number of exited members who rejoined and resu...,,,,
1,,Male,Female,Others,Not \nAvailable,Total,Male,Female,Others,Not \nAvailable,Total,Male,Female,Others,Not \nAvailable,Total
2,"Less than 18 43,488",,"13,444",16,-,"56,948","9,121","6,269",2,443,"15,835",339,173,-,-,512
3,18-21,"16,50,881","3,05,266",365,-,"19,56,512","8,67,108","1,81,901",121,"8,657","10,57,787","51,590","10,452",7,2,"62,051"
4,22-25,"18,65,623","4,07,532",390,-,"22,73,545","15,07,374","3,64,369",156,"25,230","18,97,129","1,44,823","37,742",13,21,"1,82,599"
5,26-28,"9,19,748","1,87,227",182,-,"11,07,157","9,76,600","2,21,744",69,"20,322","12,18,735","1,33,696","28,103",4,28,"1,61,831"
6,29-35,"12,14,635","3,08,803",277,-,"15,23,715","13,75,062","3,16,507",122,"36,964","17,28,655","2,00,279","34,884",16,36,"2,35,215"
7,,"More than 35 12,28,968","3,10,224",335,-,"15,39,527","14,01,532","3,48,146",93,"59,877","18,09,648","1,53,169","27,871",12,65,"1,81,117"
8,Total,"69,23,343","15,32,496",,"1,565 -","84,57,404","61,36,797","14,38,936",563,"1,51,493","77,27,789","6,83,896","1,39,225",52,152,"8,23,325"


In [19]:
def is_monthly_epf_df(df: pd.DataFrame):
    '''Returns true if the given dataframe contains EPFO subscribers'''
    is_epf_table = any(df.iloc[:, 1].str.lower().str.contains('new EPF subscriber', na=False, case=False))
    is_monthly_table = any(df.iloc[0, :].str.contains("\w+\s\d{4}", regex=True))
    return is_epf_table and is_monthly_table

def get_epf_dfs(dfs: list[pd.DataFrame]):
    """Returns EPFO Dataframes, from the given list of dataframes"""
    return list(filter(is_monthly_epf_df, dfs))

In [20]:
epf_dfs=get_epf_dfs(tables)

In [39]:
parse_date_head(epf_dfs[0])

0   2022-04-01
Name: 0, dtype: datetime64[ns]

In [38]:
def parse_date_head(df: pd.DataFrame):
    return pd.to_datetime(
        df.iloc[:, 0],
        format="%B %Y",
        errors="coerce"
    ).dropna()
def strip_month_headline(df: pd.DataFrame):

    date_df = parse_date_head(df)
    date_row_idx = date_df.index[0]
    return df.iloc[date_row_idx+1:].reset_index(drop=True)

In [27]:
def exclude_totals_row(df: pd.DataFrame):
    return df[~df.iloc[:, 0].str.contains("total", na=False, case=False)]

In [40]:
def strip_headings(df: pd.DataFrame):
    def find_gender_header_index(df: pd.DataFrame):
        return df.index[df.iloc[:, 2].str.lower()
                        .str.contains('female', na=False, case=False)][0]

    gender_header_idx = find_gender_header_index(df)

    return df.iloc[gender_header_idx:, 1:5] \
        .reset_index(drop=True).fillna(0)

In [30]:
def prep_column_names(df: pd.DataFrame):
    df = df.rename(columns=dict(
        zip(
            df.columns,
            ["epfo_" + head.lower() for head in df.iloc[0].tolist()]
        )
    ))
    return df.drop(df.index[0]).reset_index(drop=True)

In [31]:
def cast_to_numeric(df: pd.DataFrame):
    return df.replace(",", "", regex=True) \
        .apply(pd.to_numeric, errors="coerce")

In [32]:
epf_dfs[0]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,April 2022,,,,,,,,,,,,,,,
1,Age,Number of new EPF subscribers during the month,,,,,Number of members that ceased subscribing duri...,,,,,Number of exited members who rejoined and resu...,,,,
2,,Male,Female,Others,Not \nAvailable,Total,Male,Female,Others,Not \nAvailable,Total,Male,Female,Others,Not \nAvailable,Total
3,Less than 18,"6,452","1,352",-,-,"7,804","1,184",654,-,-,"1,838",449,252,-,-,701
4,18-21,"2,12,257","55,183",-,3,"2,67,443","1,15,818","27,683",1,1,"1,43,503","99,019","19,651",-,-,"1,18,670"
5,22-25,"1,96,347","77,992",1,4,"2,74,344","2,41,411","69,975",3,3,"3,11,392","2,89,641","64,049",4,-,"3,53,694"
6,26-28,"85,038","31,955",1,4,"1,16,998","1,72,559","45,397",3,8,"2,17,967","2,19,714","45,750",8,-,"2,65,472"
7,29-35,"1,21,075","56,148",1,4,"1,77,228","2,66,806","68,781",3,61,"3,35,651","3,39,875","71,544",6,-,"4,11,425"
8,More than 35,"1,49,304","62,440",4,13,"2,11,761","2,49,613","74,780",2,94,"3,24,489","3,06,577","72,229",3,1,"3,78,810"
9,Total,"7,70,473",,"2,85,070 7",28,"10,55,578","10,47,391","2,87,270",12,167,"13,34,840","12,55,275","2,73,475",21,1,"15,28,772"


In [42]:
epf_dfs[0] \
    .pipe(strip_month_headline) \
    .pipe(exclude_totals_row) \
    .pipe(strip_headings) \
    .pipe(prep_column_names) \
    .pipe(cast_to_numeric) \
    .rename(columns={"epfo_not \navailable": "epfo_notavailable"})

,epfo_male,epfo_female,epfo_others,epfo_notavailable
0,6452,1352,NaN,NaN
1,212257,55183,NaN,3.0
2,196347,77992,1.0,4.0
3,85038,31955,1.0,4.0
4,121075,56148,1.0,4.0
5,149304,62440,4.0,13.0


In [53]:
for df in epf_dfs:
    df_date = parse_date_head(df).iloc[0].date()
    df = df.pipe(strip_month_headline) \
        .pipe(exclude_totals_row) \
        .pipe(strip_headings) \
        .pipe(prep_column_names) \
        .pipe(cast_to_numeric) \
        .rename(columns={"epfo_not \navailable": "epfo_notavailable"})
    df.to_csv(f"{df_date}.csv", index=False)

c:\Users\Biky Roy\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\internals\blocks.py:2323: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)
c:\Users\Biky Roy\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\internals\blocks.py:2323: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)
c:\Users\Biky Roy\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\internals\blocks.py:2323: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)
c:\Users\Biky Roy\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\internals\blocks.py:2323: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)
c:\Users\Biky Roy\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\internals\blocks.py:2323: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)
c:\Users\Biky Roy\AppData\Loca

'2022-04-01'